In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


/home/kirell/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [3]:
# LOCAL imports
import rts
import rts.pipeline
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text

LOG = rts.utils.get_logger()

In [42]:
# vidx = rts.metadata.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.metadata.read_all_metadata(METADATA, vidx)
# rts.metadata.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.metadata.load_metadata_hdf5(METADATA, 'rts_metadata')
# rts.metadata.export_metadata_stats(df, OUTDIR)
df.head()

,guid,mediaId,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,...,collection,publishedBy,rights,title,resume,geoTheme,resumeSequence,created,published,sequences
0,AA1101000650,ZE003073,/mnt/rts/3/7/0/ZE003073,2932,16:9,SD,2010-10-10T20:05:00Z,Programme,Programme,Autres (information),...,Mise au point,TSR 1,Restriction/Condition,PETITS MEURTRES ENTRE AMIS // UN LIT POUR DEUX...,"* SUISSE : 20101010, l'inspecteur HEIMO enquêt...",[SUISSE],[00:02.36\r\nPlateau : diverses présentations ...,1670505072,1286741100,"{'892779': {'guid': 'AA1101000650', 'mediaId':..."
1,AA1101000660,ZE003216,/mnt/rts/6/1/2/ZE003216,3043,16:9,SD,2010-11-14T00:00:00Z,Programme,Programme,Autres (information),...,Mise au point,TSR 1,Restriction/Condition,BERLUSCONNERIES ? // SERPENT DU LAC // L'ADIEU...,"* ITALIE : 20101114 et archives, un nouveau sc...","[VAUD, ITALIE, PLATEAU, SUISSE, SAXON, TOUR-DE...",[00:02.38\r\nPlateau : diverses présentation d...,1670505072,1289692800,"{'893397': {'guid': 'AA1101000660', 'mediaId':..."
2,AA1101000710,ZE010504,/mnt/rts/4/0/5/ZE010504,278,16:9,SD,2010-10-31T00:00:00Z,Programme,Programme,Autres (information),...,Ensemble,TSR 1,Inconnu,PRO JUVENTUTE - 10.10.31,"* SUISSE : 20101031, présentation de la Fondat...",[SUISSE],"[00:01.00\r\nSUISSE, int salle de classe : div...",1670505072,1288483200,"{'894120': {'guid': 'AA1101000710', 'mediaId':..."
3,AA1101000730,ZE003393,/mnt/rts/3/9/3/ZE003393,1922,16:9,SD,2011-01-10T00:00:00Z,Programme,Programme,Folklore et musique populaire,...,La boîte à musique,TSR 2,Inconnu,4E TROPHEE DES MUSIQUES POPULAIRES : 1ERE DEMI...,"* Moudon, Plateau : 20101218 et archives, émis...","[MOUDON, PLATEAU]","[ Moudon, diverses séquences : diverses présen...",1670505072,1294617600,"{'895543': {'guid': 'AA1101000730', 'mediaId':..."
4,AA1101000740,ZE003206,/mnt/rts/6/0/2/ZE003206,744,16:9,SD,2010-11-23T00:00:00Z,Programme,Programme,"Jeux, concours",...,Tapis rouge,TSR 1,Restriction/Condition,10.11.23,None,None,None,1670505072,1290470400,{}


In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)

In [ ]:
# Fulltext Index from HDF5
# midx.delete_all_documents()
rts.db.fulltext.index_media(df, midx)
# Wait 30 sec for full indexation
midx.get_stats().number_of_documents  # 522103

In [43]:
sample_df = rts.metadata.get_one_percent_sample(df)
# rts.metadata.export_metadata_stats(sample_df, Path(OUTDIR) / 'sample')
sample_df

,guid,mediaId,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,...,collection,publishedBy,rights,title,resume,geoTheme,resumeSequence,created,published,sequences
7,AA1101000770,ZE002100,/mnt/rts/0/0/1/ZE002100,1524,16:9,SD,2010-04-29T17:40:00Z,Programme,Programme,Emissions de conseil,...,Dolce vita,TSR 1,Restriction/Condition,CAMPING - 10.04.29,"* Plateau : 20100429, SIKI Muriel reçoit ses i...","[LIEU INDÉTERMINÉ, VAUD, PLATEAU]",[00:01.24\r\nPlateau : diverses présentations ...,1670505072,1272562800,"{'890511': {'guid': 'AA1101000770', 'mediaId':..."
55,AA1106004840,ZS009400,/mnt/rts/0/0/4/ZS009400,530,16:9,SD,2011-03-05T00:00:00Z,Programme,Programme,Autres émissions religieuses,...,Faut pas croire,TSR 1,Restriction/Condition,11.03.05,"* Plateau : 20110305, BOISSET Christophe reçoi...","[LIEU INDÉTERMINÉ, NEUVEVILLE, LA, PLATEAU]",[00:01.18\r\nPlateau : diverses présentations ...,1670505072,1299283200,"{'897154': {'guid': 'AA1106004840', 'mediaId':..."
378,AA1201021870,ZM001400,/mnt/rts/0/0/4/ZM001400,43,16:9,SD,2012-01-22T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",...,Bande Annonce,TSR 2,Inconnu,Alain Delon. 2BLS CINEMA 2201,None,None,None,1670505073,1327190400,{}
432,AA1202023530,ZT002200,/mnt/rts/0/0/2/ZT002200,4445,16:9,SD,2012-04-08T00:00:00Z,Programme,Retransmission sportive,Retransmissions en direct,...,Motocyclisme,TSR 2,Restriction/Condition,MOTO3 - GRAND PRIX DU QATAR LOSAIL 2012,None,None,None,1670505073,1333843200,{}
523,AA1203026020,ZM002200,/mnt/rts/0/0/2/ZM002200,60,16:9,HD,2012-04-07T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",...,Bande Annonce,None,Inconnu,BLS SAMEDI 0704,None,None,None,1670505073,1333756800,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521345,GE9904151069,ZB037400,/mnt/rts/0/0/4/ZB037400,3641,4:3,SD,1964-12-04T00:00:00Z,Programme,Programme,CULTURE ET CONNAISSANCE,...,Préfaces,TSR,Inconnu,RAMUZ - 64.12.04,None,[LENS (VS)],[PRESENCE DE RAMUZ\r\nPHOTOS RAMUZ - série pho...,1670505579,-160185600,"{'782440': {'guid': 'GE9904151069', 'mediaId':..."
521481,GE9905365939,ZB044500,/mnt/rts/0/0/5/ZB044500,20,4:3,SD,1999-05-21T00:00:00Z,Sujet,Sujet journal télévisé,"Téléjournal, flashes",...,Le téléjournal,TSR 1,Inconnu,99.05.21,"* Lieu indéterminé : 19990521, lettre de la co...",[LIEU INDÉTERMINÉ],[00:24.15\r\nLieu indéterminé : divers plans d...,1670505579,927244800,"{'1094808': {'guid': 'GE9905365939', 'mediaId'..."
521764,GE9909121969,ZB164800,/mnt/rts/0/0/8/ZB164800,414,4:3,SD,1999-09-30T00:00:00Z,Programme,Programme,EMISSIONS POUR ENFANTS ET ADOLESCENTS,...,Une petite place,TSR 1,Inconnu,99.09.30,None,None,None,1670505580,938649600,{}
521811,GE9909356789,ZB044600,/mnt/rts/0/0/6/ZB044600,113,4:3,SD,1999-09-03T00:00:00Z,Sujet,Sujet journal télévisé,"Téléjournal, flashes",...,Le téléjournal,TSR 1,Inconnu,99.09.03,"* Kloten : Siège de Swissair, 19990903, les em...","[ZURICH, KLOTEN]","[00:18.10\r\nZurich : itw BRUGGISSER, Philippe...",1670505580,936316800,"{'1100081': {'guid': 'GE9909356789', 'mediaId'..."


In [46]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
# remuxed = rts.pipeline.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
# print(remuxed)


rts.pipeline.process_media(input_video_folder, LOCAL_VIDEOS, compute_transcript=True, force_trans=True)

INFO:RTS:Loading spacy model: fr_core_news_lg


{'media_id': 'ZB001000', 'status': 'success', 'error': ''}

In [ ]:
rts.pipeline.simple_process_archive(sample_df[:100], LOCAL_VIDEOS, compute_transcript=True)

In [ ]:
%ls {remux_folder}

In [ ]:
locs = rts.features.text.extract_locations("pig, inde, l'europe, Ecublens, Lausanne, Crissier, Suisse")
locs

In [41]:
ts = rts.pipeline.load_all_transcripts(LOCAL_VIDEOS)

ZU032800
ZU063800
ZT028800
ZU042800
ZU022800
ZU076800
ZU069100
ZT010100
ZU075100
ZM008100
ZE002100
ZU022100
ZU074000
ZU059000
ZM018000
ZU021000
ZU023000
ZU081000
ZT027000
ZB001000
ZU028000
ZU056000
ZM016000
ZM005000
ZU059600
ZU017600
ZU057600
ZU028600
ZU074600
ZU030500
ZM016500
ZU033500
ZU066500
ZU028500
ZU008500
ZM009500
ZU035300
ZT019300
ZT030300
ZU046300
ZT010300
ZU025300
ZM011300
ZU067300
ZU047300
ZU018300
ZU059300
ZU005300
ZU081900
ZU049900
ZU043900
ZT002900
ZU063900
ZU061700
ZU065700
ZM003700
ZU010700
ZT031700
ZT007700
ZU062700
ZU041700
ZT005700
ZU009700
ZM014700
ZU026200
ZT016200
ZT025200
ZS013200
ZU013200
ZT035200
ZU075200
ZT002200
ZU082200
ZM007200
ZM002200
ZT034200
ZM004200
ZT027200
ZU079200
ZS009400
ZU068400
ZU077400
ZU062400
ZU053400
ZU037400
ZU029400
ZM001400
ZT023400


In [31]:
loc_ts = rts.features.text.find_locations_all_transcripts(ts)

In [38]:
len(loc_ts) / float(len(ts))

0.1590909090909091

In [40]:
ts

{'ZU032800': [{'t': "Plus d'alibis. La déclaration de Federica Mogherini sonne comme un mot d'ordre.",
   's': '0.00',
   'e': '5.76'},
  {'t': 'Le principal enjeu est de construire ensemble un sens de responsabilité partagé par rapport',
   's': '5.76',
   'e': '14.04'},
  {'t': "à ce qui se passe en Méditerranée, sachant qu'il n'y a pas de solution facile, mais",
   's': '14.04',
   'e': '19.96'},
  {'t': "que nous devons être responsables en tant qu'Européens de manière cohérente.",
   's': '19.96',
   'e': '24.00'},
  {'t': "La pauvreté, l'instabilité, les guerres dans différents points du globe sont les",
   's': '24.00',
   'e': '34.24'},
  {'t': 'causes profondes des migrations. Nous devons aussi nous en préoccuper.',
   's': '34.24',
   'e': '38.44'},
  {'t': "Notre priorité doit d'abord être d'améliorer les conditions de sauvetage en mer afin que",
   's': '38.44',
   'e': '49.36'},
  {'t': 'moins de personnes meurent.', 's': '49.36', 'e': '50.96'},
  {'t': "Je le répète, c'es

In [35]:
swiss_cities = rts.features.text.get_swiss_cities()

In [13]:
ts = [{'t': "pig, inde, l'europe, Ecublens, Lausanne, Crissier, Suisse"}]
rts.features.text.find_locations(ts)

INFO:RTS:Loading spacy model: fr_core_news_lg
Ecublens | Lausanne | Crissier


[{'t': "pig, inde, l'europe, Ecublens, Lausanne, Crissier, Suisse",
  'locations': 'Ecublens | Lausanne | Crissier'}]

In [36]:
swiss_cities['Zurich']

{'lat': 47.36667, 'lon': 8.55}

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))

In [ ]:
import av
import av.datasets


video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)
out_path = os.path.join(OUTDIR, 'thumbs')

content = v

print(content)

with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    # print(stream.codec_context.height)
    for i, frame in enumerate(container.decode(stream)):
        if i % 4 != 0:
            continue


    #     print(frame)
    #     print(frame.time, frame.time_base)
        # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
        frame.to_image().save(
            os.path.join(out_path, f"f{frame.pts:04d}.jpg"),
            quality=80,
        )

In [ ]:
thumbs_dir = os.path.join(OUTDIR, 'thumbs')
outfast = './notebook/fastdup'
fastdup_data = './notebook/out'

In [ ]:
import fastdup

In [ ]:

fastdup.run(input_dir=thumbs_dir, work_dir=fastdup_data, nearest_neighbors_k=8, turi_param='ccthreshold=0.90')    #main running function.
# fastdup.create_duplicates_gallery('out/similarity.csv', save_path=outfast)     #create a visual gallery of found duplicates
# fastdup.create_outliers_gallery('out/outliers.csv',   save_path=outfast)       #create a visual gallery of anomalies
# fastdup.create_components_gallery(fastdup_data, save_path=outfast)                    #create visualiaiton of connected components
# fastdup.create_stats_gallery('out', save_path=outfast, metric='blur')          #create visualization of images stastics (for example blur)
# fastdup.create_similarity_gallery('out', save_path=outfast,get_label_func=lambda x: x.split('/')[-2])     #create visualization of top_k similar images assuming data have labels which are in the folder name
# fastdup.create_aspect_ratio_gallery('out', save_path=outfast)

In [ ]:
config = fastdup.load_config(fastdup_data)
df = fastdup.galleries.do_find_top_components(work_dir=fastdup_data, input_dir=config['input_dir'], kwargs={})